In [2]:
import pandas as pd 
import numpy as np
from langdetect import detect
import seaborn as sns
import io 
from s3_credentials import *
import boto3

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

In [3]:
cd '/Users/admin/Jedha/'

/Users/admin/Jedha


In [4]:
# boto session 
#YOUR_ACCESS_KEY = 
#YOUR_SECRET_KEY = 

session = boto3.Session(aws_access_key_id= YOUR_ACCESS_KEY, 
                        aws_secret_access_key= YOUR_SECRET_KEY)

s3 = session.resource("s3")
client = session.client("s3")

In [5]:
#Pull subdatasets

# NLP pred
obj = s3.Object('jedha-fake-reviews-project', "datasets/predictions_svm_nlp.csv")
NLP_pred = pd.read_csv(io.BytesIO(obj.get()['Body'].read()), low_memory = False, index_col=0)

# Meta pred
obj = s3.Object('jedha-fake-reviews-project', "datasets/prediction_meta_data.csv")
Meta_pred = pd.read_csv(io.BytesIO(obj.get()['Body'].read()), low_memory = False, index_col=0)

#meta_pred_svc
obj = s3.Object('jedha-fake-reviews-project', "datasets/svc_predictions_meta_data.csv")
meta_pred_svc = pd.read_csv(io.BytesIO(obj.get()['Body'].read()), low_memory = False, index_col=0)



In [6]:
dataset = pd.DataFrame(dict(NLP  = NLP_pred.iloc[:,1],
                           Meta_data = Meta_pred.iloc[:,1],
                           meta_pred_svc = meta_pred_svc.iloc[:,1],
                           is_fake_review =  Meta_pred['is_fake_review']
                                )
                                )


## Logistic Regression

In [33]:
X = dataset[['NLP', 'Meta_data', 'meta_pred_svc']]
y = dataset['is_fake_review']


X = X.values
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify = y)



# Train model
model = LogisticRegression(C=1, class_weight={0:1, 1:2.1}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


model.fit(X_train, y_train)





LogisticRegression(C=1, class_weight={0: 1, 1: 2.1})

In [ ]:
# grid search class weights with logistic regression for imbalance classification

# define grid
#cw_params = [{0:1, 1:2.1}]
cw_params = [{0:1, 1:i} for i in np.arange(2.5, 3.1, 0.1)]
#C_params = np.arange(0.1, 50, 0.1)
#C_params = [0.95]
param_grid = dict(class_weight = cw_params, C = C_params)
#param_grid = dict(class_weight = cw_params)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=69)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='f1')
# execute the grid search
grid_result = grid.fit(X_test, y_test)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [34]:
print('f1 test :',f1_score(y_test, model.predict(X_test)))
print('f1 train :',f1_score(y_train, model.predict(X_train)))



f1 test : 0.7966666666666666
f1 train : 0.8008929699675666
